In [1]:
import os
import sys
import time
import h5py
import pygrib
import numpy as np
import netCDF4 as nc
from datetime import datetime, timedelta

In [2]:
sys.path.insert(0, '/glade/u/home/ksha/GAN_proj/')
sys.path.insert(0, '/glade/u/home/ksha/GAN_proj/libs/')

from namelist import *
import data_utils as du
import graph_utils as gu

In [12]:
datetime(1900, 1, 1) + timedelta(hours=1069440)

datetime.datetime(2022, 1, 1, 0, 0)

In [ ]:
ERA5_CAPE_2019.nc       ERA5_PWAT_2022.nc
ERA5_CAPE_2020.nc       ERA5_rh_2021_bottom_layers.nc
ERA5_CAPE_2021.nc       ERA5_t2m_2021_bottom_layers.nc
ERA5_CAPE_2022.nc       ERA5_u_2021_bottom_layers.nc
ERA5_PWAT_2019.nc       ERA5_v_2021_bottom_layers.nc
ERA5_PWAT_2020.nc


In [ ]:
BATCH_dir = '/glade/campaign/cisl/aiml/ksha/BATCH_ERA5/'

year = 2021

size_mrms = 128
size_half = 64
gap = 64 # samples will have at least 5 non-overlap grids on height and width dimensions 
thres_dry = 1.0 # 1.0 mm = rain on that grid
thres_rain_cover = 0.33 # at least 25% of the grids should have rain
thres_land_cover = 0.90 # at least 90% of the grids should stay within land
thres_extreme = 30 # if a grid has 20 mm/hr or above, the sample will bypass "thres_rain_cover"
thres_extreme_cover = 0.10 # if a grid has 20 mm/hr or above, the sample will bypass "thres_rain_cover"

In [ ]:
base = datetime(year, 1, 1)
date_list = [base + timedelta(hours=day) for day in range(365*24)]

In [ ]:
with h5py.File(save_dir+'MRMS_ERA5_domain.hdf', 'r') as h5io:
    land_mask_MRMS = h5io['land_mask_MRMS'][...]
    elev_MRMS = h5io['elev_MRMS'][...]
    indx = h5io['MRMS_to_ERA5_indx'][...]
    indy = h5io['MRMS_to_ERA5_indy'][...]

In [ ]:
elev_MRMS[elev_MRMS<0] = 0
elev_MRMS = elev_MRMS/2000